In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [19]:
csv_file = "Resources/phs_2020_1.csv"
ow2020_1_df = pd.read_csv(csv_file)
csv_file2 = "Resources/phs_2020_2.csv"
ow2020_2_df = pd.read_csv(csv_file2)
csv_file3 = "Resources/phs_2021_1.csv"
ow2021_1_df = pd.read_csv(csv_file3)
#load first csv into a dataframe. Drops two columns and removes all rows != to the value All Heros
ow2020_1_df = ow2020_1_df.drop(['tournament_title','map_type'], axis=1)
ow2020_1_df = ow2020_1_df.loc[lambda ow2020_1_df: ow2020_1_df['hero_name']=='All Heroes']
#load second csv into a dataframe. Drops two columns and removes all rows != to the value All Heros
ow2020_2_df = ow2020_2_df.drop(['tournament_title','map_type'], axis=1)
ow2020_2_df = ow2020_2_df.loc[lambda ow2020_2_df: ow2020_2_df['hero_name']=='All Heroes']
#combines the first two dataframes together to create one larger dataframe
ow2020_combined_df = pd.concat([ow2020_1_df,ow2020_2_df], ignore_index=True)
#load third csv into a dataframe. Drops two columns and removes all rows != to the value All Heros
ow2021_1_df = ow2021_1_df.drop(['tournament_title','map_type'], axis=1)
ow2021_1_df = ow2021_1_df.loc[lambda ow2021_1_df: ow2021_1_df['hero_name']=='All Heroes']
#combines the 1 larger dataframe and newer one to become one larger dataframe
ow_combined_df = pd.concat([ow2020_combined_df,ow2021_1_df], ignore_index=True)
#changed column names to match Gary's dataframe column names
ow_combined_df=ow_combined_df.rename(columns={'esports_match_id':'match_id', 'player_name':'player','team_name':'team','hero_name':'hero'})
ow_combined_df=ow_combined_df.set_index('match_id')
# since there were alot of row with alot of repeated values, we used pivot table to turn the stats column and it's row values
#to many new columns to better visualize the data. Since all the data in these columns is numerical and NaN value was set to 0
owpivot = ow_combined_df.pivot_table('stat_amount',['start_time','match_id','map_name','player','team','hero'], 'stat_name', fill_value=0).reset_index()
#this code removed all the spaces from the new column
colnames=owpivot.columns
newcol=[]
finalnewcol=[]
for col in colnames:
    newcol.append(col.replace(" ","_"))
for col in newcol:
    finalnewcol.append(col.lower())
owpivot =  owpivot.set_axis(finalnewcol,axis=1)
owpivot = owpivot.drop(['knockback_kills'], axis=1)
owpivot = owpivot.rename(columns={'ultimates_earned_-_fractional':'ultimates_earned_fractional','damage_-_quick_melee':'damage_quick_melee'})
csv_file = "Resources/phs_2018_playoffs.csv"
ol2018_df = pd.read_csv(csv_file)
csv_file = "Resources/phs_2019_playoffs.csv"
ol2019_df = pd.read_csv(csv_file)
combo_df = pd.concat([ol2018_df, ol2019_df], ignore_index=True)
clean_combo_df = combo_df.drop(columns=['map_type', 'stage'])
all_hero_df = clean_combo_df.loc[lambda clean_combo_df: clean_combo_df['hero']=='All Heroes']
ol_pivot = all_hero_df.pivot_table('stat_amount',['start_time','match_id','map_name','player','team','hero'], 'stat_name', fill_value=0).reset_index()
colnames=ol_pivot.columns
newcol=[]
finalnewcol=[]
for col in colnames:
    newcol.append(col.replace(" ","_"))
for col in newcol:
    finalnewcol.append(col.lower())
ol_pivot = ol_pivot.set_axis(finalnewcol,axis=1)
ol_pivot = ol_pivot.rename(columns={'ultimates_earned_-_fractional':'ultimates_earned_fractional','damage_-_quick_melee':'damage_quick_melee'})
total_ow = pd.concat([ol_pivot, owpivot], ignore_index=True)
t = total_ow.shape[0]
val = []
x=0
while x < t:
    val.append(x+1)
    x+=1
total_ow['primary_key'] = val
total_ow=total_ow.set_index('primary_key')

In [32]:
total_ow[['start_time','time']] = total_ow.start_time.str.split(" ",expand=True)

In [35]:
total_ow = total_ow.drop(['time'], axis=1)

In [36]:
total_ow['start_time'] = pd.to_datetime(total_ow['start_time'])

In [38]:
rds_connection_string = "postgres:postgres@localhost:5432/overwatchleague_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [39]:
engine.table_names()

['overwatch_db']

In [40]:
total_ow.to_sql(name='overwatch_db', con=engine, if_exists='append', index=True)

In [41]:
pd.read_sql_query('select * from overwatch_db', con=engine).head()

,primary_key,start_time,match_id,map_name,player,team,hero,all_damage_done,assists,average_time_alive,...,teleporter_pads_destroyed,time_alive,time_building_ultimate,time_elapsed_per_ultimate_earned,time_holding_ultimate,time_played,turrets_destroyed,ultimates_earned_fractional,ultimates_used,weapon_accuracy
0,1,2018-07-12,13121.0,Dorado,Boombox,Philadelphia Fusion,All Heroes,13492.01651,12.0,138.299004,...,0.0,715.853022,493.406014,100.804086,249.378007,761.135303,0.0,4.894703,3.0,0.217886
1,2,2018-07-12,13121.0,Dorado,Gamsu,Boston Uprising,All Heroes,10677.17934,10.0,60.004185,...,0.0,661.745020,538.363016,107.885988,188.237006,761.135303,0.0,4.990111,3.0,0.486804
2,3,2018-07-12,13121.0,Dorado,HOTBA,Philadelphia Fusion,All Heroes,15229.71661,21.0,161.522338,...,0.0,737.272022,540.146016,74.337529,216.135006,761.135303,0.0,7.266128,12.0,0.364929
3,4,2018-07-12,13121.0,Dorado,Kellex,Boston Uprising,All Heroes,80.00000,0.0,44.717001,...,0.0,677.276020,484.036014,87.737222,210.131006,761.135303,0.0,5.516883,5.0,0.333333
4,5,2018-07-12,13121.0,Dorado,Mistakes,Boston Uprising,All Heroes,18116.51992,4.0,100.819378,...,0.0,674.050020,651.524019,76.553131,108.765003,761.135303,0.0,8.510743,7.0,0.395415
